In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

In [2]:
"""
File Reading
"""
a = []
for i in range(14,31):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except Exception as e:
                    print(e)
                    print(file.name)
    except:
        print("10/"+str(i))
for i in range(1,30):
    if i != 10:
        try:
            for j in range(100):
                with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                    try:
                        a.append(str(file.read()))
                    except Exception as e:
                        print(file.name)
        except:
            print("11/"+str(i))
print(len(a))

10/19
10/20
10/26
10/27
11/2
11/3
11/9
11/16
11/17
11/22
11/23
11/24
3300


In [3]:
"""
Neural Network Model Creation
"""

"""
Based on code from https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b
"""
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
x_train = np.array(a, dtype=np.str)
x_train = x_train.astype(str)
# print(x_train.shape)
y_train = np.zeros(2700)+0.9
print(y_train.shape)
import tensorflow as tf
embed_size = 300 
max_features = 50000 
maxlen = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
model1 = tf.keras.Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(2,activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model1.summary()
x_trainR = np.array(x_train)[:2200]
y_trainR = np.array(y_train)[:2200]
x_test = np.array(x_train)[2200:]
y_test = np.array(y_train)[2200:]
x_train = x_trainR
y_train = y_trainR
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

(2700,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          439296    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 15,474,290
Trainable params: 15,474,290
Non-trainable params: 0
____________________________________

In [4]:
"""
Formats data labels
"""
def toFinal(a):
    for i in range(len(a)):
        if a[i] > 0:
            a[i] = 1
        else:
            a[i] = 0

In [5]:
"""
Reads and formats data labels
"""
y_train = []
for i in range(14,32):
    try:
        with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla.csv") as file:
            j = file.read().split(',')[1]
            assert j is not None
            j = j.replace('\n','')
            for _ in range(100):
                y_train.append(float(j))
    except:
        print("Bad: 10/"+str(i))
for i in range(1,22):
    try:
        with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla.csv") as file:
            j = file.read().split(',')[1]
            assert j is not None
            j = j.replace('\n','')
            for _ in range(100):
                y_train.append(float(j))
    except:
        print("Bad: 11/"+str(i))
toFinal(y_train)
# print(y_train)
y_train = np.array(y_train)
# print(y_train)
print(y_train.shape)
print(x_train.shape)


Bad: 10/19
Bad: 10/20
Bad: 10/23
Bad: 10/25
Bad: 10/26
Bad: 10/27
Bad: 11/2
Bad: 11/3
Bad: 11/9
Bad: 11/10
Bad: 11/16
Bad: 11/17
(2700,)
(2200, 100)


In [6]:
print(np.count_nonzero(y_train))
print(y_train.size)
y_train = y_train[:1500]
x_train = x_train[:1500]

1700
2700


In [7]:
"""
Trains Model
"""
history = model1.fit(x_train, y_train, epochs=200, batch_size=100)

Train on 1500 samples
Epoch 1/200
1500/1500 [==============================] - 8s 5ms/sample - loss: 0.6940 - accuracy: 0.5333
Epoch 2/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6908 - accuracy: 0.5333
Epoch 3/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6889 - accuracy: 0.5393
Epoch 4/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6851 - accuracy: 0.5347
Epoch 5/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6814 - accuracy: 0.5627
Epoch 6/200
1500/1500 [==============================] - 7s 5ms/sample - loss: 0.6786 - accuracy: 0.5460
Epoch 7/200
1500/1500 [==============================] - 8s 5ms/sample - loss: 0.6745 - accuracy: 0.5753
Epoch 8/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6676 - accuracy: 0.5800
Epoch 9/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6665 - accuracy: 0.5567
Epoch 10/200
1500/1500 [=========

1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6388 - accuracy: 0.5867
Epoch 79/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6388 - accuracy: 0.5880
Epoch 80/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6384 - accuracy: 0.5873
Epoch 81/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6382 - accuracy: 0.5793
Epoch 82/200
1500/1500 [==============================] - 7s 5ms/sample - loss: 0.6387 - accuracy: 0.5880
Epoch 83/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6391 - accuracy: 0.5920
Epoch 84/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6390 - accuracy: 0.5853
Epoch 85/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6384 - accuracy: 0.5853
Epoch 86/200
1500/1500 [==============================] - 6s 4ms/sample - loss: 0.6384 - accuracy: 0.5953
Epoch 87/200
1500/1500 [==============================] - 6

KeyboardInterrupt: 

In [10]:
def predict(x):
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(x))
    x =pad_sequences(tokenizer.texts_to_sequences(x), maxlen=maxlen)
    return model1.predict(x)

In [ ]:
print(model1.weights)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
acc = history.history['loss']
plt.plot(acc)
